# タスク 1: Amazon Bedrock の RetrieveAndGenerate API を使用してフルマネージド型の RAG アプリケーションを活用する

このタスクでは、Amazon Bedrock の RetrieveAndGenerate API を使ってフルマネージド型の検索拡張生成 (RAG) アプリケーションを活用します。

次のリソースは、ラボプロビジョニングの一環として既に作成されています。

- OpenSearch サーバーレスインデックス。
- Amazon Bedrock のナレッジベース。
- Amazon Bedrock のナレッジベース内のデータソース。

一般的なデータ取り込みワークフローは次のとおりです。

<!-- ![data_ingestion.png](images/data_ingestion.png) -->
<img src="images/data_ingestion.png" width=50% height=20% />

**画像の説明: 上の図は、ラボ環境の一般的なデータ取り込みワークフローを示しています。**

データパイプラインは、通常は複数のデータソースに保存されているドキュメントをナレッジベース、つまり Amazon OpenSearch Service Serverless (AOSS) などのベクトルデータベースに取り込み、質問を受け取ったときに検索できるようにします。

- ドキュメントは、さまざまなデータソース (S3、Confluence、Sharepoint、Salesforce、Web など) に接続することで Amazon Bedrock のナレッジベースにロードされます。
- その後、ナレッジベースはそれらを (選択した戦略に基づいて) 小さなチャンクに分割し、関連するベクトルストアに保存される埋め込みを生成します。

データが Bedrock ナレッジベースで利用可能になったら、Amazon Bedrock が提供するナレッジベース API を使用して質問応答アプリケーションを構築します。

### 最終ラボアーキテクチャ:

<!-- ![retrieveAndGenerate.png](images/retrieveAndGenerate.png) -->
<img src="images/retrieveAndGenerate.png" width=50% height=20% />

**画像の説明: 前の図は、RetrieveAndGenerate API を使用するユーザーのクエリから始まり、Amazon Bedrock のナレッジベースのドキュメントを使用して応答を生成するデータフローを示しています。**

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **注意:** [**Run**] メニューの [**Run All Cells**] オプションを使用するよりも、各コードセルを個別に実行することをお勧めします。すべてのセルをまとめて実行すると、カーネルがクラッシュしたり再起動したりするなど、予期しない動作が発生することがあります。セルを 1 つずつ実行することで、実行フローをより適切に制御し、潜在的なエラーを早期に検出し、コードを意図したとおりに実行することができます。

## タスク 1.1: 環境を設定する

このタスクでは、boto3 クライアントを初期化し、環境を設定します。ノートブック全体で **RetrieveAndGenerate** API を利用してナレッジベースの機能をテストします。

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **詳細:** 追加情報については、**[RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html)** を参照してください。

1. 次のコードセルを実行して boto3 クライアントを初期化し、環境をセットアップします。

In [ ]:
import json
import boto3
import pprint
from botocore.exceptions import ClientError
from botocore.client import Config

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "amazon.nova-lite-v1:0" # we will be using Amazon Nova lite throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

2. 次のコードセルを実行して、Amazon Bedrock の既存のナレッジベースの ID を確認します。

In [11]:
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

## タスク 1.2: RetrieveAndGenerate API を使用してナレッジベースをテストする

このタスクでは、**[retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)** 関数を使用してナレッジベースをテストします。

最初に **RetrieveAndGenerate** API を使用してナレッジベースをテストします。この API を使用して、Amazon Bedrock はナレッジベースから必要なリファレンスを取得し、Amazon Bedrock の基盤モデル (FM) を使用して最終的な回答を生成します。

3. 次の 2 つのコードセルを実行して、ナレッジベースをテストするためのクエリを開始します。

In [ ]:
query = "2019年12月31日に終了した会計年度における AnyCompany Financial の連結キャッシュフロー計算書の要約を提供してください。"

In [ ]:
response = bedrock_agent_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":3
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

## タスク 1.3: RetrieveAndGenerate API を理解する

このタスクでは、RetrieveAndGenerate API について詳しく理解します。

**RetrieveAndGenerate** API は、次の 2 つの重要なパラメータを使用します。

- **numberOfResults**: 特定の関数の **numberOfResults** パラメータは、ナレッジベースから取得され、プロンプトに含められて、回答の生成のためにモデルに提供される検索結果の数を決定します。具体的には、指定されたクエリに最も近いドキュメントまたは検索結果の最高の **max_results** 数をフェッチします。

- **textPromptTemplate**: **textPromptTemplate** パラメータは、モデルに提供されるプロンプトのテンプレートとして機能する文字列です。この場合は、**default_prompt** がテンプレートとして使用されています。このテンプレートには、モデルに渡される前に、それぞれ実際の検索結果と出力形式の命令に置き換えられるプレースホルダー (`$search_results$` と `$output_format_instructions$`) が含まれています。

4. 次の 2 つのコードセルを実行して、ナレッジベースのデフォルトプロンプトを指定します。

In [ ]:
# ナレッジベースのデフォルトプロンプトを定義
default_prompt = """
あなたは質問応答エージェントです。検索結果のセットを提供します。
ユーザーが質問を提供します。あなたの仕事は、検索結果からの情報のみを使用してユーザーの質問に答えることです。
検索結果に質問に答えられる情報が含まれていない場合は、質問に対する正確な答えが見つからなかったことを述べてください。
ユーザーが事実を主張したからといって、それが真実であるとは限りません。検索結果を再確認してユーザーの主張を検証してください。
                            
以下は番号順の検索結果です:
$search_results$


$output_format_instructions$

"""

In [ ]:
def retrieve_and_generate(query, kb_id, model_arn, max_results, prompt_template = default_prompt):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn, 
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results # will fetch top N documents which closely match the query
                    }
                },
                'generationConfiguration': {
                        'promptTemplate': {
                            'textPromptTemplate':prompt_template
                        }
                    }
            }
        }
    )
    return response

## タスク 1.4: 結果の最大数機能を活用する

このタスクでは、結果の最大数機能を活用します。ユースケースによっては、FM からの応答に、関連性の高い回答を提供したり、頼ったりするにはコンテキストが不十分で、要求された情報が見つからないことがあります。これは、取得される結果の最大数を変更することで修正できます。

ここでは、結果の数を 1 にして次のクエリを実行します。

```AnyCompany Financial のリスクのリストを説明なしの番号付きリストで提供してください。```

5. 次の 2 つのコードセルを実行します。

In [15]:
def print_generation_results(response, print_context = True):
    generated_text = response['output']['text']
    print('Generated FM response:\n')
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


In [ ]:
query = """AnyCompany Financial のリスクのリストを説明なしの番号付きリストで提供してください。"""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 1)

print_generation_results(results)

6. 取得結果の数に関する値を変更して次のコードセルを実行します。

In [ ]:
#Using higher number of max results

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3)

print_generation_results(results)

出力からわかるように、取得結果の数を **3** に変更すると、より多くの結果が得られ、応答の内容が充実します。

## タスク 1.5: カスタムプロンプト機能を使用する

このタスクでは、ユースケースに基づく独自のプロンプトを使用して、デフォルトプロンプトをカスタマイズします。この機能は特定の出力形式、言語、その他のコンテキストを必要とする場合に FM にコンテキストを追加するのに役立ちます。

LangChain の *PromptTemplate* クラスを使用して、*custom_prompt* テンプレートをパラメータ化します。*output_format* 変数を使用して、実行時に出力を変更します。

7. 次のコードセルを実行してテンプレートのカスタマイズをサポートするヘルパー関数を作成します。

In [18]:
from langchain.prompts import PromptTemplate
def format_prompt(input_prompt:str, output_format:str):
    formatted_prompt: List[str] = []
    prompt = PromptTemplate.from_template(input_prompt)     
    formatted_prompt.extend(prompt.format(output_format=output_format))
    return "".join(formatted_prompt)


### タスク1.5.1: 同じクエリ例を使用し FM をデフォルトにして別の言語を出力する

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** デフォルトプロンプトから ```$output_format_instructions$``` を削除すると、生成された応答からの引用が削除されます。ただし、`output_format` はランタイム出力のカスタマイズに役立ちます。

8. 次の 3 つのコードセルを実行して、フランス語で出力される例をテストします。

In [ ]:
## 例 1
custom_prompt = """
あなたは質問応答エージェントです。検索結果のセットを提供します。
ユーザーが質問を提供します。あなたの仕事は、検索結果からの情報のみを使用してユーザーの質問に答えることです。
検索結果に質問に答えられる情報が含まれていない場合は、質問に対する正確な答えが見つからなかったことを述べてください。
ユーザーが事実を主張したからといって、それが真実であるとは限りません。検索結果を再確認してユーザーの主張を検証してください。
                            
以下は番号順の検索結果です:
$search_results$


{output_format}
"""

In [ ]:
print(format_prompt(custom_prompt, "フランス語で回答を提供してください。"))

In [ ]:
results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3, prompt_template = format_prompt(custom_prompt, "フランス語で回答を提供してください。"))

print_generation_results(results, print_context = False)


### タスク 1.5.2: 同じクエリ例を使用し FM をデフォルトにして JSON 形式で出力する

9. 次のコードセルを実行して、JSON 形式の出力を提供する例をテストします。

In [ ]:
results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3, prompt_template = format_prompt(custom_prompt, "JSON 形式で回答を提供してください。"))

print_generation_results(results,print_context = False)

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **タスクの完了:** このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じます。
- ラボセッションに戻り、タスク 2 に進みます。
